## 这是用来测试 粒子群 算法拟合路径的文件

In [1]:
import pandas as pd
import numpy as np
from pyswarm import pso
from shapely import Point, LineString
import geopandas as gpd
import ast

In [2]:
data_path = '1m_precision_staiLine.csv'
swath_path = '../Scratch/test_Load_Shp/test_shps/swath_group_1.shp'
raw_data = pd.read_csv(data_path)
real_swaths = gpd.read_file(swath_path)

In [3]:
# 处理 raw_data
raw_data['height'] = raw_data['height'].apply(ast.literal_eval)
raw_data['aspect'] = raw_data['aspect'].apply(ast.literal_eval)
raw_data['slope'] = raw_data['slope'].apply(ast.literal_eval)
data_array = raw_data.to_numpy()
data_array

array([[440345.9340906528, 4397532.697626984,
        list([20617, 20577, 20558, 20601, 20581, 20551, 20594, 20578, 20549]),
        list([136.49759, 125.403076, 127.29251, 103.172554, 98.36588, 104.03625, 89.43273, 97.74034, 106.4293]),
        list([7.704224, 10.512283, 6.76528, 6.8809066, 7.343308, 5.0072403, 7.1958723, 7.403187, 5.804205]),
        0],
       [440346.4306018925, 4397531.829596691,
        list([20581, 20551, 20548, 20578, 20549, 20534, 20565, 20545, 20530]),
        list([98.36588, 104.03625, 125.942116, 97.74034, 106.4293, 119.81416, 106.18921, 105.395546, 107.47478]),
        list([7.343308, 5.0072403, 3.53399, 7.403187, 5.804205, 3.668241, 8.038443, 5.1120825, 4.0478396]),
        0],
       [440346.9271131323, 4397530.961566399,
        list([20581, 20551, 20548, 20578, 20549, 20534, 20565, 20545, 20530]),
        list([98.36588, 104.03625, 125.942116, 97.74034, 106.4293, 119.81416, 106.18921, 105.395546, 107.47478]),
        list([7.343308, 5.0072403, 3.53399,

In [4]:
# 适应度函数
def fitness_function(lambdas, origin_x, origin_y, height, aspect, slope, labels):
    aspect = np.deg2rad(aspect)
    x_off = lambdas[0] * np.cos(aspect) * slope
    y_off = lambdas[1] * np.sin(aspect) * slope
    
    # print('slope')
    # print(slope)
    # print('aspect')
    # print(aspect)
    # print('origin xy')
    # print(origin_x)
    # print(origin_y)
    # print('off xy')
    # print(x_off)
    # print(y_off)
    # print('lambdas shape')
    # print(lambdas)
    
    distances = []
    # new_x= origin_x + x_off
    new_x = np.add(origin_x, x_off)
    # new_y = origin_y + y_off
    new_y = np.add(origin_y, y_off)
    # print(new_x)
    # print(new_y)
    temp_point = Point(new_x, new_y)
    temp_real_swath = real_swaths.geometry.iloc[labels]
    distances = temp_point.distance(temp_real_swath)
    # for x, y, label in zip(new_x, new_y, labels):
    #     temp_point = Point(x, y)
    #     temp_real_swath = real_swaths.geometry.iloc[label]
    #     distances.append(temp_point.distance(temp_real_swath))
    return np.mean(distances)

In [5]:
# 读取参数
origin_x = data_array[:, 0]
origin_y = data_array[:, 1]
height = data_array[:, 2]
aspect = data_array[:, 3]
slope = data_array[:, 4]
labels = data_array[:, 5]
[len(origin_x), len(origin_y), len(height), len(aspect), len(slope), len(labels)]

[2428, 2428, 2428, 2428, 2428, 2428]

In [6]:
# 设定 lambdas 上下界
lower_bound = [-1, -1]
upper_bound = [1, 1]
# 用 pso 进行优化
# args = (origin_x, origin_y, height, aspect, slope, labels)
# x_optimize, f_optimize = pso(fitness_function, lb=lower_bound, ub=upper_bound, args=args)
# 处理数据
results = []
for i in range(len(raw_data)):
    # 提取当前数据点的输入
    origin_x = raw_data.loc[i, 'origin_x']
    origin_y = raw_data.loc[i, 'origin_y']
    height = raw_data.loc[i, 'height'][4]
    aspect = raw_data.loc[i, 'aspect'][4]
    slope = raw_data.loc[i, 'slope'][4]
    label = raw_data.loc[i, 'label']

    # 使用 PSO 进行优化
    args = (origin_x, origin_y, height, aspect, slope, label)  # 需要传入适应度函数的额外参数
    # print(args)
    xopt, fopt = pso(fitness_function, lower_bound, upper_bound, args=args)

    # 将结果存储起来
    results.append((xopt, fopt))

Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: maximum

KeyboardInterrupt: 